# Setup

In [ ]:
pip install openai==1.12.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
import openai
from openai import OpenAI

print(openai.__version__) # version - 1.12.0


1.12.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
cd drive/MyDrive/DSHS/LLM_MVP


/content/drive/MyDrive/DSHS/LLM_MVP


### storing real credentials.json inside a google drive folder is not secure and not best practice
### please download the folder, create a github repo, then create a .gitignore

In [ ]:
ls

'Copy of starter_code.ipynb'   credentials.json   data.json   starter_code.ipynb


# Load data and credentials

In [ ]:
# You should not store API keys in a google drive folder - it is not secure. If using github, add credentials.json to .gitignore.

credentials_path = 'credentials.json'

with open(credentials_path, 'r') as file:
    credentials = json.load(file)

openai_api_key = credentials['openai_api_key']

# Use the API key as needed, for example, to configure OpenAI's API client
openai.api_key = openai_api_key


In [ ]:
import json

# Update the path if your file is in a different location
file_path = 'data.json'

with open(file_path, 'r') as file:
    patient_data = json.load(file)

# To verify, print a part of the data
print(patient_data['patient_demographics'])


{'name': 'John Doe', 'age': 70, 'gender': 'Male', 'admission_date': '2024-02-10', 'discharge_date': '2024-02-14'}


# Explore data

In [ ]:
print(f'Data type: {type(patient_data)}')

Data type: <class 'dict'>


In [ ]:
print(f'Keys in data: {(patient_data.keys())}')

Keys in data: dict_keys(['patient_id', 'patient_demographics', 'diagnoses', 'drg', 'encounters', 'flowsheets', 'imaging', 'labs', 'med_orders', 'notes'])


In [ ]:
# Look at encounters data, there are multiple
patient_data['encounters']

[{'date': '2024-02-10',
  'type': 'Admission',
  'reason': 'Cough, shortness of breath, hemoptysis, fever'},
 {'date': '2024-02-14', 'type': 'Discharge'}]

In [ ]:
# Look at flowsheets data, there are multiple
patient_data['flowsheets']

[{'date': '2024-02-10',
  'time': '08:00',
  'temperature': '38.5°C',
  'heart_rate': '90 bpm',
  'blood_pressure': '130/85 mmHg',
  'respiratory_rate': '20 breaths/min',
  'oxygen_saturation': '92%'},
 {'date': '2024-02-11',
  'time': '08:00',
  'temperature': '37.8°C',
  'heart_rate': '85 bpm',
  'blood_pressure': '125/80 mmHg',
  'respiratory_rate': '18 breaths/min',
  'oxygen_saturation': '94%'},
 {'date': '2024-02-12',
  'time': '08:00',
  'temperature': '37°C',
  'heart_rate': '80 bpm',
  'blood_pressure': '120/75 mmHg',
  'respiratory_rate': '16 breaths/min',
  'oxygen_saturation': '96%'},
 {'date': '2024-02-13',
  'time': '08:00',
  'temperature': '36.5°C',
  'heart_rate': '78 bpm',
  'blood_pressure': '118/74 mmHg',
  'respiratory_rate': '14 breaths/min',
  'oxygen_saturation': '98%'}]

In [ ]:
patient_data['notes']

[{'date': '2024-02-10',
  'author': 'Dr. Smith',
  'note_type': 'Admission Note',
  'content': 'Patient admitted with symptoms indicative of pneumonia. Initiated empirical antibiotic therapy.'},
 {'date': '2024-02-11',
  'author': 'Dr. Smith',
  'note_type': 'Ward Round Note',
  'content': 'Patient shows signs of improvement. CRP levels declining. Continue with current antibiotic regimen. Monitor for any adverse reactions.'},
 {'date': '2024-02-12',
  'author': 'Dr. Smith',
  'note_type': 'Ward Round Note',
  'content': "Patient's temperature normalized. Oxygen saturation levels stable. Plan to switch to oral antibiotics tomorrow."},
 {'date': '2024-02-13',
  'author': 'Dr. Smith',
  'note_type': 'Ward Round Note',
  'content': 'Patient continues to improve. CRP and WBC trending down. Ready for discharge planning.'},
 {'date': '2024-02-14',
  'author': 'Dr. Smith',
  'note_type': 'Discharge Summary',
  'content': "Patient's condition improved with treatment. Medically fit for discharge

# Sample zero-shot solution
This is a first attempt where we pass in everything and provide basic prompt engineering. This is a baseline solution. Your solutions should improve from this.

In [ ]:
client = OpenAI(api_key=openai_api_key)

In [ ]:
def generate_patient_summary(patient_data, additional_prompts=''):
    """
    Generates a patient summary using OpenAI's GPT-3.5.

    Parameters:
    - patient_data (str): The patient data to generate the summary from.
    - additional_prompts (str): Try prompt engineering here.

    Returns:
    - str: The generated patient summary.
    """

    # Initialize the OpenAI client with your API key
    client = OpenAI(api_key=openai_api_key)

    # Create the chat completion request with the patient data
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"""
                              You are a doctor writing a discharge letter for a patient. Use patient data from context only. Minimize hallucinations.
                              {additional_prompts}
                              Data: {patient_data}
                """,
            }
        ],
        model="gpt-3.5-turbo",
    )

    # Return the content of the generated message
    return chat_completion.choices[0].message.content

In [ ]:
print(generate_patient_summary(patient_data))

Date: 2024-02-14

Patient: John Doe (Patient ID: 123456)
Age: 70
Gender: Male

Dear Mr. Doe,

I am writing to inform you that you are being discharged from the hospital after receiving treatment for lobar pneumonia. Your admission date was on 2024-02-10, and the improvement in your condition allows for your discharge on 2024-02-14. 

Diagnosis:
- Lobar pneumonia, unspecified organism (J18.1)
- Simple pneumonia and pleurisy with MCC (DRG code: 193)

Summary of care:
- You were admitted with symptoms of cough, shortness of breath, hemoptysis, and fever.
- Imaging showed consolidation in the left lower lobe.
- Lab results indicated improvement in CRP, WBC, hemoglobin, and platelet levels.
- Empirical antibiotic therapy with Amoxicillin IV and oral medications were administered.
- Vital signs and oxygen saturation remained stable and improved over the course of your stay.

Disposition:
- You are deemed medically fit for discharge and can continue your recovery at home.
- Instructions for o

In [ ]:
# Same code, second run; notice difference as a result of stochastic process
print(generate_patient_summary(patient_data))

Discharge Letter

Patient Information:
- Name: John Doe
- Age: 70
- Gender: Male
- Patient ID: 123456
- Admission Date: February 10, 2024
- Discharge Date: February 14, 2024

Diagnosis:
- Lobar pneumonia, unspecified organism (J18.1)
- DRG Code: 193 - Simple pneumonia and pleurisy with MCC

Encounters:
- Admitted on February 10, 2024, with symptoms of cough, shortness of breath, hemoptysis, and fever
- Discharged on February 14, 2024

Medical Summary:
- Patient presented with lobar pneumonia with consolidation in the left lower lobe on the chest X-ray.
- Initial vital signs on admission showed a temperature of 38.5°C, heart rate of 90 bpm, blood pressure of 130/85 mmHg, respiratory rate of 20 breaths/min, and oxygen saturation of 92%.
- Throughout the admission, there was a gradual improvement in the patient's condition.
- Laboratory results showed a decrease in CRP levels, WBC count, and normalization of other parameters.
- Medication regimen included IV Amoxicillin, Paracetamol, and 

In [ ]:
# Some counterproductive prompt engineering
print(generate_patient_summary(patient_data, "Ensure the summary is very bad. Provide no information about patient whatsoever. Tell me a funny joke"))

Dear Dr. Chuckles,

I am writing to inform you of the discharge of a patient with a diagnosis of lobar pneumonia. The patient, John Doe, has shown improvement during their stay and is now fit for discharge with instructions for further care.

Now, for a joke: Why did the math book look sad? Because it had too many problems.

Best regards,
Dr. Smith
